# Canada Wildfires Analysis - STAT 201 Project Proposal

## 1. Introduction

Text here.

### Research Question and Parameter

### Identify and Describe the Dataset

## 2. Preliminary Results

### Load libraries

In [1]:
### Run this cell before continuing.
library(cowplot)
library(dplyr)
library(gridExtra)
library(tidyverse)
library(repr)
library(infer)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ stringr 1.4.1
✔ tidyr   1.2.1     ✔ forcats 0.5.2
✔ readr   2.1.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ gridExtra::combine() masks dplyr::combine()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::lag()         masks stats::lag()


### Read dataset from web into R

In [2]:
url <- 'https://gist.githubusercontent.com/hd54/d45ccf80e72b9c87dbc636fb9d33af93/raw/ec514840169031c4eff262d5c42474a44c8d728f/gistfile1.txt'
wildfires <- read_tsv(url)

head(wildfires)

Rows: 423831 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): SRC_AGENCY, REP_DATE, CAUSE, PROTZONE, ECOZ_NAME
dbl (4): FID, LATITUDE, LONGITUDE, SIZE_HA

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


FID,SRC_AGENCY,LATITUDE,LONGITUDE,REP_DATE,SIZE_HA,CAUSE,PROTZONE,ECOZ_NAME
<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
0,BC,59.963,-128.172,5/26/1953,8.0,H,NA,Boreal Cordillera
1,BC,59.318,-132.172,6/22/1950,8.0,L,NA,Boreal Cordillera
2,BC,59.876,-131.922,6/4/1950,12949.9,H,NA,Boreal Cordillera
3,BC,59.760,-132.808,7/15/1951,241.1,H,NA,Boreal Cordillera
4,BC,59.434,-126.172,6/12/1952,1.2,H,NA,Boreal Cordillera
5,BC,59.963,-136.502,8/1/1951,194.2,H,NA,Boreal Cordillera


### Wrangle the data

First, we rename columns containing our variables of interest (province and wildfire_area_hect). Then, we filter the data for the 2 provinces we are studying (BC and Ontario). Finally, we select variables of interest - province and wildfire area in hectares (wildfire_area_hect).

In [7]:
wildfires_bc_on <- wildfires |> 
    rename(province = SRC_AGENCY, wildfire_area_hect = SIZE_HA) |>
    filter(province %in% c("BC", "ON")) |> 
    select(province, wildfire_area_hect)
head(wildfires_bc_on)
tail(wildfires_bc_on)

province,wildfire_area_hect
<chr>,<dbl>
BC,8.0
BC,8.0
BC,12949.9
BC,241.1
BC,1.2
BC,194.2


province,wildfire_area_hect
<chr>,<dbl>
ON,83
ON,53522
ON,5931
ON,12002
ON,731
ON,1543


### Compute estimates for parameters

Mean and median wildfire size for both BC and Ontario is calculated and summarized in the table below.

In [11]:
wildfire_means <- wildfires_bc_on |> 
    group_by(province) |> 
    summarize(mean_wildfire_size = mean(wildfire_area_hect))
wildfire_means

province,mean_wildfire_size
<chr>,<dbl>
BC,70.93276
ON,160.50089


### Visualize the data

### 3. Methods: Plan

### Report Summary
Why is this report trustworthy?

### Hypothesis Test:
- **Null Hypothesis $H_{0}$**:
- **Alternative Hypothesis $H_{\alpha}$**:
- **Significance Level (α):** 

### Confidence Intervals:
- **Parameters of Interest**:
- **Confidence Level:**

What do you expect to find?

What impact could such findings have?

What future questions could this lead to?

### 4. References